# 导入库文件

In [ ]:
import json
import re
from openai import OpenAI
import datetime
import logging
import os
from tqdm import tqdm
import time
import pandas as pd
from http import HTTPStatus
import os
import dashscope

In [ ]:
with open('path_data/real_data.txt','r',encoding='utf-8') as f:
    all_student_data = f.read()
all_student_data = eval(all_student_data)
all_stu_set = pd.DataFrame(all_student_data)
all_student_data = all_stu_set.to_dict(orient='records')
all_stu_set.head()

In [ ]:
def get_kp_info(kp_list):
    all_kp_info = []
    for one_kp in kp_list:
        one_kp_info={'id':one_kp['id'],'掌握度':round(one_kp['master'],4),'知识点名称':one_kp['name']}
        all_kp_info.append(one_kp_info)
    return all_kp_info

In [ ]:
def get_video_info(video_list):
    all_video_info = []
    for one_video in video_list:
        one_video_info = {'id':one_video['re_id'],'课程名称':one_video['kp_0'],
                          '视频关联知识点':one_video['kp_3'],'视频平均学习时长':one_video['video_t_mean']}
        all_video_info.append(one_video_info)
    return all_video_info

# 定义智能体

In [ ]:
def get_prompt(student_data,prompt_type='Analytics',ana_report='',path_reflect='',learn_path=[],is_CL=True,is_ZPD=True):
    """
    student_data: 学生信息；prompt_type：提示词类型Analytics,Planning,Planning2,Reflection；
    is_CL：是否加载认知负荷模块；is_ZPD：是否加载最近发展区模块
    """
    stu_id = student_data['stu_id']
    video_t_mean = round(student_data['video_t_mean'],2)
    risk_week = student_data["risk_week"]
    course_name = student_data["course_name"]
    course_about = student_data["course_about"]
    kp_weak_list = get_kp_info(student_data["kp_weak_list"])
    kp_master_list = get_kp_info(student_data["kp_master_list"])
    kp_unlearned_list = get_kp_info(student_data["kp_unlearned_list"])
    stu_info = f"""
            ### 学生的信息如下：
            - **学生ID**：{stu_id};
            - **学生每个视频的平均学习时长(video_t_mean)**：{video_t_mean};
            - **课程名称(course_name)**：{course_name};
            - **课程时长**：30周;
            - **课程介绍(course_about)**：{course_about};
            - **学生薄弱知识点**：{kp_weak_list}
            - **学生已掌握知识点**：{kp_master_list}
            - **学生未学习知识点**：{kp_unlearned_list}
            """
    
    # Cognitive Load Constraints
    # Zone of Proximal Development Constraints
    video_list = get_video_info(student_data["video_list_rec"])
    video_info =  f"""{video_list}"""
    CL_prompt = """### 认知负荷约束如下：
                """
    
    ZPD_prompt = """### 最近发展区约束如下：
                 """
    
    role_cl_zpd = ''
    if is_CL is True and is_ZPD is True:
        role_cl_zpd = '需特别关注**认知负荷**和**最近发展区**的约束条件。'
    if is_CL is True and is_ZPD is False:
        role_cl_zpd = '需特别关注**认知负荷**的约束条件。'
    if is_CL is False and is_ZPD is True:
        role_cl_zpd = '需特别关注**最近发展区**的约束条件。'
    if is_CL is False:
        CL_prompt = ""
    if is_ZPD is False:
        ZPD_prompt = ""
    if ana_report!='':
        ana_report = f'### 学情分析报告如下：\n{ana_report}'
    if path_reflect!='':
        path_reflect = f'修改意见如下：\n{path_reflect}'
    if learn_path!=[]:
        learn_path = f'### **已有学习路径如下：**\n{learn_path}'
    # 这里是路径规划智能体，是不是应该有两个呀？一个是初始路径规划，一个是路径完善？
    
    Learning_Analytics_prompt = f"""
             ## 智能体角色设定：
             你是一个经验丰富的学情分析专家，在课程第{risk_week}周有一名面临学业风险的学生他需要你的帮助。以下是学生的相关信息：


            ## 任务目标：
                请根据以下步骤生成学情分析报告：
            
            ## 输入信息：
                {stu_info}
            
            ## 输出格式如下：
            {{
              "学情报告": {{
                "整体评估": "……",
                "学情概述": "……"
              }}
            }}
            
            ## 注意事项
                1. 请确保分析结果基于提供的学生信息，逻辑清晰，数据准确。
                2. 请使用友好、鼓励的语言风格，帮助学生树立信心。
                3. 请严格按照 JSON 格式返回结果，确保字段完整且格式正确。
            """
    
    Path_Planning_prompt = f"""
            ## 智能体角色设定：
            你是一名经验丰富的学习路径规划专家。在课程第{risk_week}周有一位学生被监测出存在较高的学业风险，需要你的帮助。
            请你根据以下给定的信息，结合学生的学情分析报告，为其规划合理的学习路径。
            {role_cl_zpd}
            
            ## 任务目标：
            5. {CL_prompt}
            6. {ZPD_prompt}
            

            ## 输入信息：
            {stu_info}
            {ana_report}

            ### **备选视频列表信息如下：**
            {video_info}

            ## 输出格式如下：
            {{
                "学习路径": [
                    {{
                        "资源 ID": "具体的资源 ID",
                        "阶段": "第一步",
                        "推荐理由": "以指导教师的口吻，第二人称详细说明为什么这个资源适合学生当前的学习需求，以及如何帮助学生改进知识掌握。",
                        "视频平均学习时长（秒）": "具体视频时长"
                    }},
                    {{
                        "资源 ID": "具体的资源 ID",
                        "阶段": "第二步",
                        "推荐理由": "以指导教师的口吻，第二人称详细说明此资源如何在难度上适合学生并帮助他实现知识的递进掌握。",
                        "视频平均学习时长（秒）": "具体视频时长"
                    }}
                    // 可以继续添加更多步骤...
                ],
                "整体学习路径推荐理由": "详细说明整体路径设计的逻辑，确保学习路径的有效性与合理性。"
            }}
            ## 注意事项
                1. 请确保分析结果基于提供的信息，逻辑清晰，数据准确。
                2. 请严格按照 JSON 格式返回结果，确保字段完整且格式正确。
            """
    
    Path_Reflection_prompt = f"""
            ## 智能体角色设定：
            你是一个经验丰富的学习路径评估专家。在课程第{risk_week}周有一名面临学业风险的学生需要你的帮助，他在第{risk_week}周就被检测出存在学业风险。
            你需要根据学生的情况评估给出的学习路径是否符合要求。
            {role_cl_zpd}
            
            ## 任务目标:
            评估已有的学习路径是否符合下面的要求。
            3、{CL_prompt}
            4、{ZPD_prompt}
            
            ## 输入信息:
            {stu_info}
            {learn_path}

            ## 输出格式如下:
            {{'is_good': 1 or 0,'修改意见':'如果is_good是0表示该路径需要修改，请给出修改建议, 如果没有问题则 is_good为 1。'}}
            
            ## 注意事项:
                1. 请确保分析结果基于提供的信息，逻辑清晰，数据准确。
                2. 请严格按照 JSON 格式返回结果，确保字段完整且格式正确。
            """
    if prompt_type=='Analytics':
        one_prompt = Learning_Analytics_prompt
    if prompt_type=='Planning':
        one_prompt = Path_Planning_prompt
#     if prompt_type=='Planning2':
#         one_prompt = Path_Planning_prompt_2
    if prompt_type=='Reflection':
        one_prompt = Path_Reflection_prompt
    return one_prompt

In [ ]:
def get_one_output(client,messages):
    print(f'【消息长度:{get_content_len(messages)}】')
    if MODEL_NAME != 'llama3.3-70b':
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
        )
        model_output = response.choices[0].message.content
     
    
    if MODEL_NAME == 'llama3.3-70b':
        response = dashscope.Generation.call(
            api_key='sk-2f4ce9fa99d440b3afaed48e1d1c73db',
            model='llama3.3-70b-instruct',
            messages=messages,
            result_format='message',  
            )
        if response.status_code == HTTPStatus.OK:
            model_output = response.output.choices[0].message.content
            print('response正常')
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            
    print('【token_use:】',response.usage)
    is_json = True
    try:
        json_result = re.search(r'{.*}', model_output, re.DOTALL).group()
        final_result = json.loads(json_result)
    except Exception as e:
        final_result = model_output
        is_json = False
        print("【解析失败，返回原始结果:】")
    return {'is_json':is_json,'final_result':final_result,
            'model_output':model_output,'token_use':dict(response.usage)}

def get_content_len(msgs):
    all_len = 0
    for one in msgs:
        all_len += len(one['content'])
    return all_len

# 路径规划

In [ ]:
MODEL_LIST = ['gpt-3.5-turbo','gpt-4o',
              'ecnu-plus','qwen-plus',
              'deepseek-v3','llama3.3-70b','claude-3-5-sonnet-20241022'
             ]
MODEL_NAME = 'ecnu-plus'
ana_report=''
path_reflect=''
learn_path=[]
is_CL=True
is_ZPD=True
config_llm = {'ecnu-plus': 
              {'base_url':'https://chat.ecnu.edu.cn/open/api/v1',
               'api_key':''},
             'gpt-3.5-turbo': 
              {'base_url':'https://api.pumpkinaigc.online/v1',
               'api_key':''},
              'gpt-4o': 
              {'base_url':'https://api.gptsapi.net/v1',
               'api_key':''},
              'claude-3-5-sonnet-20241022': 
              {'base_url':'https://api.gptsapi.net/v1',
               'api_key':''},
              'qwq': 
              {'base_url':'http://127.0.0.1:11434/v1/',
               'api_key':'ollama'},
              'qwen-plus':
              {'base_url':'https://dashscope.aliyuncs.com/compatible-mode/v1',
               'api_key':''},
              'deepseek-v3':
              {'base_url':'https://dashscope.aliyuncs.com/compatible-mode/v1',
               'api_key':''},
              'llama3.3-70b':
              {'base_url':'https://dashscope.aliyuncs.com/compatible-mode/v1',
               'api_key':''}
             }

client = OpenAI(base_url=config_llm[MODEL_NAME]['base_url'],
                api_key=config_llm[MODEL_NAME]['api_key'])

time_now = datetime.datetime.now()
now_str = time_now.strftime('%Y-%m-%d-%H-%M-%S')
log_dir = f'log/path_plan/{MODEL_NAME}'
log_file_name = f'{log_dir}/{now_str}-{is_CL}-{is_ZPD}.txt'
os.makedirs(log_dir, exist_ok=True)

ALL_STU_PATH_GET = []
Process_data = []
REF_NUM = 1
print('运行信息：',{'MODEL_NAME':MODEL_NAME,'is_CL':is_CL,'is_ZPD':is_ZPD,'REF_NUM':REF_NUM})
stu_num = 0
call_api_num = 0
for student_data in all_student_data:  
    one_stu_id = student_data['stu_id']
    stu_num += 1
    print(f'【进度 {stu_num} / {len(all_student_data)}】')
    start_time = time.time()
    one_process_data = {}
    one_process_data['MODEL_NAME'] =MODEL_NAME
    one_process_data['is_CL'] =is_CL
    one_process_data['is_ZPD'] =is_ZPD
    one_process_data.update(student_data.copy())
    Learning_Analytics_prompt = get_prompt(student_data,prompt_type='Analytics',learn_path=[],is_CL=is_CL,is_ZPD=is_ZPD)
    # 日志保存
    one_process_data['Learning_Analytics_prompt'] = Learning_Analytics_prompt
    print('处理学生：',one_stu_id)
    Analytics_msgs= [{"role": "system", "content": "You are a helpful assistant."},
                     {"role": "user", "content": Learning_Analytics_prompt}]
    print('>>学情分析中...')
    Analytics_out = get_one_output(client,Analytics_msgs)
    call_api_num += 1
    one_process_data['Analytics_out'] = Analytics_out
    if Analytics_out['is_json']:
        ana_report = Analytics_out['final_result']
        Path_Planning_prompt = get_prompt(student_data,prompt_type='Planning',
                                          ana_report=ana_report['学情报告']['整体评估'],
                                          is_CL=is_CL,is_ZPD=is_ZPD)
        one_process_data['Path_Planning_prompt'] = Path_Planning_prompt
        Planning_msgs = [{"role": "system", "content": "You are a helpful assistant."},
                                 {"role": "user", "content": Path_Planning_prompt}]
        print('>>路径规划中...')
        Planning_out = get_one_output(client,Planning_msgs)
        call_api_num += 1
        one_process_data['Planning_out_0'] = Planning_out
        if Planning_out['is_json']:
            one_learn_path = Planning_out['final_result']
            is_good_path = False
            path_refection_comment = ''
            for ref_num in range(REF_NUM):
                Path_Reflection_prompt = get_prompt(student_data,prompt_type='Reflection',
                                          learn_path=one_learn_path['学习路径'],
                                          is_CL=is_CL,is_ZPD=is_ZPD)
                one_process_data[f'Path_Reflection_prompt_{ref_num}'] = Path_Reflection_prompt
                Path_Reflection_msgs = [{"role": "system", "content": "You are a helpful assistant."},
                                        {"role": "user", "content": Path_Reflection_prompt}]
                print('>>路径反思中...')
                Reflection_out = get_one_output(client,Path_Reflection_msgs)
                call_api_num += 1
                one_process_data[f'Reflection_out_{ref_num}'] = Reflection_out
                if Reflection_out['is_json']:
                    path_refection = Reflection_out['final_result']
                    if int(path_refection['is_good'])==1:
                        print('【路径通过】')
                        break
                    else:
                        one_process_data['Reflection_num'] = ref_num+1
                        path_refection_comment = path_refection['修改意见']
                        print('【**触发反思**】', path_refection['修改意见'])
                        # 开始重新规划路线
                        Planning_msgs.append({"role": "system", "content": Planning_out['model_output']})
                        Planning_msgs.append({"role": "user", "content": path_refection_comment})
                        one_process_data[f'path_refection_comment_{ref_num}'] = path_refection_comment
                        print('>>路径优化中...')
                        Planning_out = get_one_output(client,Planning_msgs)
                        call_api_num += 1
                        one_process_data[f'Planning_out_{ref_num}'] = Planning_out
                        if Planning_out['is_json']:
                            one_learn_path = Planning_out['final_result']
                        else:
                            print('>>完善路径失败跳出循环...')
                            break
                else:
                    break
        else:
            continue
    else:
        continue
    print('【学习路径输出】',one_learn_path)
    end_time = time.time()
    run_time = end_time - start_time
    print('运行时间：',run_time)
    one_process_data['run_time'] = run_time
    Process_data.append(one_process_data)
    ALL_STU_PATH_GET.append({'stu_id':one_stu_id,'learn_path':one_learn_path})
    with open(log_file_name,'w',encoding='utf-8') as f:
        f.write(str(Process_data))
    log_file_name_pkl = f'{log_dir}/{now_str}-{is_CL}-{is_ZPD}.pkl'
    pd.to_pickle(Process_data,log_file_name_pkl)
#     break
print('call_api_num: ',call_api_num)
print('日志：',log_file_name)

# 计算性能

In [ ]:
stu_load = []
rec_load = []
all_v_len = []
all_path_t = []
for one_stu in ALL_STU_PATH_GET:
    one_stu_id = int(one_stu['stu_id'])
    for one in all_student_data:
        if one['stu_id'] == one_stu_id:
            one_res_list = one['video_list_rec']
            break
    one_v_load = []
    for one_rec_v in one_stu['learn_path']['学习路径']:
        one_t = 0
        if '资源 ID' in one_rec_v.keys():
            for one_o_v in one_res_list:
                one_t = one_o_v['video_t_mean']
                one_rec_id = -11
                try:
                    one_rec_id = int(one_rec_v['资源 ID'])
                except:
                    print('资源 ID 生成错误',one_rec_v['资源 ID'])
                if one_o_v['re_id'] == one_rec_id:
                    one_t = one_o_v['video_t_mean']
                    one_v_load.append(one_t)
                    break
            if one_t == 0:
                print('无法查询资源 ID',one_rec_v['资源 ID'])
        else:
            print('json生成错误，无资源 ID',one_rec_v)
    if len(one_v_load) != 0:
        # 在这里计算k_oder的计算？
        all_v_len.append(len(one_v_load))
        all_path_t.append(one_v_load)
        rec_load.append(np.mean(one_v_load))
        for one in all_student_data:
            if one['stu_id'] == one_stu_id:
                stu_load.append(one['video_t_mean'])
                break
absolute_errors = (np.array(stu_load) - np.array(rec_load))/np.array(stu_load)
mean_absolute_error_custom = np.mean(abs(absolute_errors))
print(f"认知负荷错位率: {mean_absolute_error_custom}")
k_oder = []
for one_p in all_path_t:
    one_k_oder = []
    if len(one_p) > 1:
        for i in range(len(one_p)-1):
            one_k_oder.append(one_p[i]<one_p[i+1])
        k_oder.append(np.mean(one_k_oder))
    else:
        k_oder.append(0.4)

In [ ]:
import pandas as pd
re_data = pd.DataFrame()
re_data['absolute_errors'] =absolute_errors
re_data['APL'] = all_v_len
re_data['ALD'] = rec_load
re_data['KSC'] = k_oder
re_data['CLMR'] = abs(re_data['absolute_errors'])
re_data['MODEL_NAME'] = MODEL_NAME
re_data.to_csv(f'log/path_plan/{MODEL_NAME}_re.csv',index=False)
re_data.describe()